In [1]:
import setup
setup.init_django()

In [2]:
from blog.models import BlogPost
from sentence_transformers import SentenceTransformer

In [3]:
from openai import OpenAI
from decouple import config
EMBEDDING_MODEL= SentenceTransformer("all-MiniLM-L6-v2")
RECREATE_DATA=True

In [4]:
docs=["The dog jumped over the cat",
     'The cat jumped over the dog',
     'The weather is very hot',
    'the cat is yellow and dog is red']

In [5]:
new_data=[]
for i,x in enumerate(docs):
    new_data.append(BlogPost(title=f"Blog Post {i+1}", content=x, can_delete=True))
if RECREATE_DATA:    
    qs=BlogPost.objects.filter(can_delete=True) 
    qs.delete()
    qs=BlogPost.objects.bulk_create(new_data)

In [6]:
qs=BlogPost.objects.filter(can_delete=True)
qs.count()

4

In [7]:
def get_embedding(text,model=EMBEDDING_MODEL):
    text=text.replace("\n","").strip()
    return model.encode(text)

In [8]:
EMBEDDING_MODEL

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [9]:
for obj in qs:
    if obj.embedding is None:
        obj.embedding=get_embedding(obj.get_embedding_text_raw())
        obj.save()
    

In [17]:
query="The dog jumped over the green cow"
# query="The dog jumped over the cat"
query_embedding=get_embedding(query)

In [18]:
from pgvector.django import CosineDistance
from django.db.models import F
qs=BlogPost.objects.annotate(distance=CosineDistance('embedding',query_embedding),
                             similarity=1-F("distance")).order_by("distance")[:2]
for obj in qs:
    print(obj.title, obj.distance, obj.similarity*100)

Blog Post 1 0.4009067355775705 59.90932644224295
Blog Post 2 0.4123959366201141 58.76040633798859
